In [10]:
from tqdm import tqdm
import logging

import pandas as pd

from openai import AzureOpenAI

# 配置日志记录
logging.basicConfig(filename='gpt4o_process.log', level=logging.INFO, encoding="utf-8", filemode='w',
                    format='%(asctime)s:%(levelname)s:%(message)s')
prompt= """后面的“待分析文本”是一段师生对话，其中，学生话语已经剔除，只保留老师话语，请对老师的话语进行分析，具体分析方法如下所示：  
将”待分析文本“分割成”发起“、”评价“、”讲解“、“其它”四种子文本段，”发起“的分割尽可能细一点。“发起”是老师邀请、引导、鼓励学生用话语来回应的语句；“评价”是对学生回应的表扬、认可、批评等评价性话语；”讲解“是老师针对知识展开描述或对学生回应的总结；不能归属于上面三种子文本段，归属为“其它”。
按照下面“示例”输出： 
示例： 老师话语： 它分别种了什么树呢？谁来说说？于凯，你来说说看。你慢讲啊。嗯，然后呢？ 
输出： 
[{"type":"发起"，"content":"它分别种了什么树呢？谁来说说？"} 
{"type":"发起"，"content":"于凯，你来说说看。"} 
{"type":"其它"："content":"你慢讲啊。嗯，"}
{"type":"发起"："content":"然后呢？"}]
待分析文本："""

# 注意：这个部署在azure上的api接口会在2025年三月份的时候过期

client = AzureOpenAI(
    azure_endpoint="https://zonekey-gpt4o.openai.azure.com/",
    api_key="b01e3eb073fe43629982b30b3548c36e",
    api_version="2024-02-01"
)

df = pd.read_excel("726四分类法.xlsx")
for i in tqdm(range(len(df))):
    try:
        response = client.chat.completions.create(
            model="soikit_test",  # model = "deployment_name".
            response_format={"type": "json_object"},  # 响应格式为JSON对象
            messages=[
                {"role": "system", "content":  "你是一个乐于助人的小助手,并且输出是一个json，key是result"},
                {"role": "user", "content": prompt+df.loc[i, 'text']},
            ]
        )
        output = response.choices[0].message.content
        df.loc[i, 'predict'] = output
        logging.info(f"Row {i} processed successfully. Output: {output}")
    except Exception as e:
        df.loc[i, 'predict'] = "error"
        logging.error(f"Error processing row {i}: {e}")




100%|██████████| 11/11 [02:18<00:00, 12.61s/it]


In [11]:
df

,Unnamed: 0.1,Unnamed: 0,text,predict
0,0,0,请袁艺喊“开始上课”，声音要响亮啊。同学们好，请坐。真棒。昨天呀，我们一起学习了课文八。今天...,"{\n ""result"": [\n {""type"": ""发起"", ""content""..."
1,1,3,嗯，细心。还有呢？他每天都去浇，天天都去浇水施肥。说明他怎么样？对，非常勤劳。猴子是不是很勤...,"{\n ""result"": [\n {""type"": ""其它"", ""content""..."
2,2,4,好的，你连叫两声。猴哥，给我听听。哎，猴哥听见了。来，我们学着他的样子。小乌鸦们，叫两声猴哥...,"{\n ""result"": [\n {""type"": ""发起"", ""content""..."
3,3,6,哪知小乌鸦再来试试看，再来劝劝猴哥。你来，欢迎陈一坤。哎，这只小乌鸦讲的好不好？非常棒，真好...,"{\n ""result"": [\n {""type"":""发起"", ""content"":..."
4,4,7,真的是男孩子今天读的真好，而且很多女孩子也在认真听，我也要表扬，非常好啊。来，同学们，你们来...,"{\n ""result"": [\n {""type"": ""评价"", ""content""..."
5,5,8,好，把三句话来说一说。我再问乌鸦对猴子说的最后一句话是什么。好，喜鹊对猴子说的最后一句话。杜...,"{""result"":[\n {""type"":""发起"",""content"":""好，把三句..."
6,6,9,同桌如果讲喜趣的话呢？那要反过来讲。等一会下课再出去，啊？是不是讲得差不多了？来，小乌鸦来试...,"{\n ""result"": [\n {""type"": ""发起"", ""co..."
7,7,10,能够把故事讲用一个。第二个能加上动作。来来，第三个能不能表演得更好？哎，加上表情啊。你来试试...,"{\n ""result"": [\n {\n ""type"": ""发起"",\n..."
8,8,11,不行，你按照老师的要求去做。请坐。要批评。你等会儿三十秒的话都到我这来。一定要积极思考，不是...,"{\n ""result"": [\n {\n ""type"": ""其它"",\n..."
9,9,12,乌鸦说过了，咱们不重复。你不着急吗？意思就是说你会着急的，是不是啊？意思一样啊。好，很好，掌...,"{\n ""result"": [\n {""type"": ""其它"", ""content""..."


In [13]:
df.to_excel('726四分类法.xlsx')

In [47]:
json.loads(output)

{'result': [{'type': '发起', 'content': '请袁艺喊“开始上课”，声音要响亮啊。'},
  {'type': '发起', 'content': '同学们好，请坐。'},
  {'type': '评价', 'content': '真棒。'},
  {'type': '讲解', 'content': '昨天呀，我们一起学习了课文八。今天，小猴子呀要到我们班来做客了。我们来跟他打打招呼。“猴子”。'},
  {'type': '讲解',
   'content': '昨天在写作业的时候呀，小朋友要注意哦。这个“猴”，反犬旁，旁边是一个单立人，中间有没有一个短竖啊？那么昨天在作业当中，方老师看到有人加了一竖，那就不对了，变成错别字了。明白了吗？好，现在用眼睛看，用心记住这个字，“猴子”。'},
  {'type': '讲解', 'content': '哎，每天通过学习啊，我们知道了这个猴子啊，种了一些果树。'},
  {'type': '发起', 'content': '它分别种了什么树呢？'},
  {'type': '发起', 'content': '谁来说说？'},
  {'type': '发起', 'content': '于凯，你来说说看。'},
  {'type': '其它', 'content': '你慢讲啊。嗯，'},
  {'type': '发起', 'content': '然后呢？'},
  {'type': '讲解', 'content': '然后种了杏树。'},
  {'type': '发起', 'content': '最后呢？'},
  {'type': '讲解', 'content': '最后还种了桃树。'},
  {'type': '发起', 'content': '到最后怎么样啊？'},
  {'type': '发起', 'content': '好，于凯，你能不能连起来把这句话来说一说？'},
  {'type': '发起', 'content': '“猴子种了”。'},
  {'type': '评价', 'content': '非常棒，请坐。'},
  {'type': '讲解', 'content': '我们像他一样，用一句话把这个故事，哎，就讲出来了。'},
  {'type': '发起', 'conte

In [22]:

from tqdm import tqdm
import json
import pandas as pd
df = pd.read_excel("726四分类法.xlsx")
gpt4o_output = []
manual_laebl = []
for i in tqdm(range(len(df))):
    print(i)
    gpt4o_output.extend(json.loads(df.loc[i,'predict']).get("result"))
    manual_laebl.extend(json.loads(df.loc[i,'label']))

 55%|█████▍    | 6/11 [00:00<00:00, 5939.54it/s]

0
1
2
3
4
5
6


JSONDecodeError: Extra data: line 3 column 48 (char 49)

In [ ]:
[
{"type": "发起", "content": "好，把三句话来说一说。我再问乌鸦对猴子说的最后一句话是什么。好，杜鹃对猴子说的最后一句话。"},
{"type": "发起", "content": "我再问乌鸦对猴子说的最后一句话是什么。"},
{"type": "评价", "content": "好"},
{"type": "发起", "content": "喜鹊对猴子说的最后一句话。"},
{"type": "发起", "content": "三句话意思一样吗？"},
{"type": "评价", "content": "对，一样。"},
{"type": "讲解", "content": "但是为什么这里他的说法不一样？你有这么耐心吗？你能等得及吗？你不着急吗？都在说猴子怎么样。都在说猴子怎么样。都在说猴子会着急，没耐心，等不及。是不是啊？这个猴子越来越着急，"},
{"type": "评价", "content": "是的啊。在这里他有三个不同的想法。很好，"},
{"type": "发起", "content": "你说还有什么？"},
{"type": "评价", "content": "啊，很好。"},
{"type": "讲解", "content": "他说出一个特点来。这三句为什么不一样？如果三句都一样就没意思了。读起来是不是这样子？让这个故事更丰富。很好。同样的意思可以用不同的句子来表达。这就是我们中国语言的神秘之处啊。我们中国语言文字的奥秘所在。"},
{"type": "发起", "content": "好，你说，你有什么补充？"},
{"type": "评价", "content": "好，谢谢你的发现。"},
{"type": "发起", "content": "你来说，正当他们成活的时候，是不是？"},
{"type": "评价", "content": "好，非常好。"},
{"type": "讲解", "content": "来，孩子们，这三段话居然找到相似之处。哎，那我们讲这个故事的时候，是不是觉得很好讲？"},
{"type": "评价", "content": "是的。"},
{"type": "发起", "content": "好的，自己选一个角色。你可以选乌鸦，可以选喜鹊，也可以选杜鹃。把他的这段话自己在下面练讲。等一会儿像高婷一样站起来啊，很熟练的啊，很有感情的告诉给大家。"},
{"type": "发起", "content": "好的啊，自己练好了，是不是？那可以讲给你的同桌好朋友听啊。哎，如果你是讲乌鸦的话，你讲给同桌听。"}
]


In [6]:
import re

text = '''
根据您的要求，以下是对老师话语的分析：

[
  {"type": "发起", "content": "请袁艺喊“开始上课”，声音要响亮啊。"},
  {"type": "其它", "content": "同学们好，请坐。真棒。昨天呀，我们一起学习了课文八。今天，小猴子呀要到我们班来做客了。我们来跟他打打招呼。“猴子”。"},
  {"type": "讲解", "content": "昨天在写作业的时候呀，小朋友要注意哦。这个“猴”，反犬旁，旁边是一个单立人，中间有没有一个短竖啊？那么昨天在作业当中，方老师看到有人加了一竖，那就不对了，变成错别字了。明白了吗？好，现在用眼睛看，用心记住这个字，“猴子”。"},
  {"type": "发起", "content": "哎，每天通过学习啊，我们知道了这个猴子啊，种了一些果树。它分别种了什么树呢？谁来说说？"},
  {"type": "发起", "content": "于凯，你来说说看。"},
  {"type": "其它", "content": "你慢讲啊。嗯，"},
  {"type": "发起", "content": "然后呢？"},
  {"type": "讲解", "content": "然后种了杏树。最后呢？最后还种了桃树。到最后怎么样啊？"},
  {"type": "发起", "content": "好，于凯，你能不能连起来把这句话来说一说？“猴子种了”。"},
  {"type": "评价", "content": "非常棒，请坐。"},
  {"type": "讲解", "content": "我们像他一样，用一句话把这个故事，哎，就讲出来了。来，我们一起来试试看。“猴子种了”。"},
  {"type": "评价", "content": "瞧，咱们一句话就能把故事的意思讲出来了。小朋友们真能干。"},
  {"type": "讲解", "content": "好，那我们来看看哦，他为什么什么都没种成呢？我们来读课文的第一段。"},
  {"type": "发起", "content": "来，谁来读？那等一下啊。好，你来读。“猴子”。"},
  {"type": "其它", "content": "嗯嗯，你先不要坐下，这个“天天”说明什么呀？"}
]
'''

matches = re.findall(r'\[.*\]', text, re.DOTALL)

print(type(matches[0]))


<class 'str'>


In [8]:
import json
json.loads(matches[0])

[{'type': '发起', 'content': '请袁艺喊“开始上课”，声音要响亮啊。'},
 {'type': '其它',
  'content': '同学们好，请坐。真棒。昨天呀，我们一起学习了课文八。今天，小猴子呀要到我们班来做客了。我们来跟他打打招呼。“猴子”。'},
 {'type': '讲解',
  'content': '昨天在写作业的时候呀，小朋友要注意哦。这个“猴”，反犬旁，旁边是一个单立人，中间有没有一个短竖啊？那么昨天在作业当中，方老师看到有人加了一竖，那就不对了，变成错别字了。明白了吗？好，现在用眼睛看，用心记住这个字，“猴子”。'},
 {'type': '发起', 'content': '哎，每天通过学习啊，我们知道了这个猴子啊，种了一些果树。它分别种了什么树呢？谁来说说？'},
 {'type': '发起', 'content': '于凯，你来说说看。'},
 {'type': '其它', 'content': '你慢讲啊。嗯，'},
 {'type': '发起', 'content': '然后呢？'},
 {'type': '讲解', 'content': '然后种了杏树。最后呢？最后还种了桃树。到最后怎么样啊？'},
 {'type': '发起', 'content': '好，于凯，你能不能连起来把这句话来说一说？“猴子种了”。'},
 {'type': '评价', 'content': '非常棒，请坐。'},
 {'type': '讲解', 'content': '我们像他一样，用一句话把这个故事，哎，就讲出来了。来，我们一起来试试看。“猴子种了”。'},
 {'type': '评价', 'content': '瞧，咱们一句话就能把故事的意思讲出来了。小朋友们真能干。'},
 {'type': '讲解', 'content': '好，那我们来看看哦，他为什么什么都没种成呢？我们来读课文的第一段。'},
 {'type': '发起', 'content': '来，谁来读？那等一下啊。好，你来读。“猴子”。'},
 {'type': '其它', 'content': '嗯嗯，你先不要坐下，这个“天天”说明什么呀？'}]